<a href="https://colab.research.google.com/github/monicasjsu/deep_learning/blob/master/keras_logisticregression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, f1_score, recall_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import  train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
import tensorflow as tf
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from keras.models import Sequential
from keras.layers import Dense,Dropout, Flatten
from keras.optimizers import Adam,sgd
from sklearn.metrics import mean_absolute_error
from keras import layers, metrics
import keras
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

Using TensorFlow backend.


In [2]:
!pip install PyDrive

In [0]:
df = pd.read_csv("/content/drive/My Drive/Datasets/dl_2/adult.csv")

In [4]:
df.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [0]:
df = df[df.workclass != '?']

In [6]:
df

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K
5,34,Private,216864,HS-grad,9,Divorced,Other-service,Unmarried,White,Female,0,3770,45,United-States,<=50K
6,38,Private,150601,10th,6,Separated,Adm-clerical,Unmarried,White,Male,0,3770,40,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,22,Private,310152,Some-college,10,Never-married,Protective-serv,Not-in-family,White,Male,0,0,40,United-States,<=50K
32557,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32558,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32559,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K


In [0]:
df = df[df.occupation !='?']

In [0]:
for cols in ['age', 'workclass', 'fnlwgt', 'education', 'education.num', 'marital.status', 'occupation','relationship', 'race', 'sex','capital.gain', 'capital.loss','hours.per.week','native.country','income']:
    df[cols] = df[cols].astype('category')
    df[cols] = df[cols].cat.codes

In [9]:
df.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
1,65,2,6157,11,8,6,3,1,4,0,0,89,17,39,0
3,37,2,6632,5,3,0,6,4,4,0,0,88,39,39,0
4,24,2,15952,15,9,5,9,3,4,0,0,88,39,39,0
5,17,2,13423,11,8,0,7,4,4,0,0,87,44,39,0
6,21,2,7370,0,5,5,0,4,4,1,0,87,39,39,0


In [10]:
df.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education.num     0
marital.status    0
occupation        0
relationship      0
race              0
sex               0
capital.gain      0
capital.loss      0
hours.per.week    0
native.country    0
income            0
dtype: int64

In [11]:
df.income.unique()

array([0, 1], dtype=int8)

In [12]:
output=df['income']
input = df.drop(['income'],axis=1)
output

1        0
3        0
4        0
5        0
6        0
        ..
32556    0
32557    0
32558    1
32559    0
32560    0
Name: income, Length: 30718, dtype: int8

In [0]:
input=input.values
output=output.values

In [14]:
print(output)

[0 0 0 ... 1 0 0]


In [15]:
print(input)

[[   65     2  6157 ...    89    17    39]
 [   37     2  6632 ...    88    39    39]
 [   24     2 15952 ...    88    39    39]
 ...
 [   23     2  7633 ...     0    39    39]
 [   41     2  7451 ...     0    39    39]
 [    5     2 12189 ...     0    19    39]]


In [0]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(input,output,test_size=0.2,random_state=0)

In [17]:
print(X_train)
print(X_test)

[[   10     2  5763 ...     0    39    39]
 [   43     2  4181 ...     0    39    39]
 [   33     2   829 ...     0    39    39]
 ...
 [   15     2 15059 ...     0    39    26]
 [    9     2  2827 ...     0    39    39]
 [   18     4  8941 ...     0    59    39]]
[[   24     2 12720 ...     0    39    39]
 [    4     2 14485 ...     0    39    39]
 [   31     5  9081 ...     0    49    39]
 ...
 [   46     2 17213 ...     0    39    39]
 [   35     2  9832 ...     0    39    39]
 [   37     1 18969 ...     0    39    26]]


In [18]:
!pip3 install 'gast==0.2.2'
try:
  %tensorflow_version 2.x
except Exception:
  pass

In [0]:
def model(X_train,X_test,y_train,y_test):
    keras_model = Sequential()
    keras_model.add(Dense(300, activation="relu",input_shape=(14,)))
    keras_model.add(Dropout(0.3))
    keras_model.add(Dense(10, activation="relu"))
    keras_model.add(Dropout(0.1))
    keras_model.add(Dense(4, activation="relu"))
    keras_model.add(Dense(1,activation='sigmoid'))
    keras_model.compile(loss='binary_crossentropy', optimizer=sgd(), metrics=['accuracy'])
    keras_model.fit(X_train, y_train, epochs=100)
    a = keras_model.evaluate(X_test, y_test)
    return a

In [20]:
model(X_train,X_test,y_train,y_test)

Epoch 1/100
24574/24574 [==============================] - 1s 50us/step - loss: 11495729844635778.0000 - accuracy: 0.7489
Epoch 2/100
24574/24574 [==============================] - 1s 45us/step - loss: 0.5631 - accuracy: 0.7506
Epoch 3/100
24574/24574 [==============================] - 1s 45us/step - loss: 26827922185.1243 - accuracy: 0.7506
Epoch 4/100
24574/24574 [==============================] - 1s 46us/step - loss: 0.5617 - accuracy: 0.7506
Epoch 5/100
24574/24574 [==============================] - 1s 44us/step - loss: 0.5617 - accuracy: 0.7506
Epoch 6/100
24574/24574 [==============================] - 1s 44us/step - loss: 0.5617 - accuracy: 0.7506
Epoch 7/100
24574/24574 [==============================] - 1s 45us/step - loss: 0.5617 - accuracy: 0.7506
Epoch 8/100
24574/24574 [==============================] - 1s 44us/step - loss: 0.5617 - accuracy: 0.7506
Epoch 9/100
24574/24574 [==============================] - 1s 48us/step - loss: 0.5617 - accuracy: 0.7506
Epoch 10/100
24574/2

[0.5598198304263254, 0.7522786259651184]